In [108]:
import pymysql
import pandas as pd
import random
import numpy as np
from sshtunnel import SSHTunnelForwarder
import json

 
# 假设有一个名为data.json的文件
json_file_path = 'table_config.json'
 
# 读取JSON文件
with open(json_file_path, 'r', encoding='utf-8') as file:
    datas = json.load(file)
    
split_info = datas['table-config']['tables'][1]['split_tables']

source_table = datas['table-config']['tables'][0]['real_table_name']


In [109]:
conn1 = pymysql.connect(host='172.28.72.225', user='root', port=7487,
                       password='l84kCG5KP4uNNtRX', database='dgut')

In [110]:

server = SSHTunnelForwarder(
    ('172.28.72.225', 22), # 指定 ssh 登录的跳转机的 address，端口号
    ssh_username='root', # 远程服务器的用户名，注意不是 DB 的用户名和密码
    ssh_password='GDS&123softswitch', # 远程服务器的密码
    remote_bind_address=('172.28.160.99', 13306), # 注意端口号不要加引号
    # local_bind_address=('127.0.0.1', 3306)#端口可以自己设置，没有冲突即可，不加这个参数也可
)
server.start()

# 数据库表是com_tran_info userlog 的sada_gdpi_click
conn2 = pymysql.connect(host='127.0.0.1',
                port=server.local_bind_port, # server.local_bind_port, 
                user='duser',
                password='RAkj#7853',
                database='userlog1',
                connect_timeout=10,
            )

In [116]:
cur1 = conn1.cursor()
cur2 = conn2.cursor()

In [112]:
query_ins = "SELECT * FROM sada_gdpi_click_dtl_dgut_ins"
cur2.execute(query_ins)
description = cur2.description
ins_results = cur2.fetchall()

In [117]:
ins_results

((910001,
  '192.168.1.3',
  'Ad2',
  1695201000,
  'http://example.org',
  'http://anotherref.com',
  'Mozilla/5.0',
  '192.168.1.4',
  'cookie_value2',
  '8081',
  '{"key": "value2"}',
  datetime.datetime(2024, 9, 24, 7, 46, 8),
  2),
 (910002,
  '192.168.1.5',
  'Ad3',
  1695202000,
  'http://example.net',
  'http://yetanotherref.com',
  'Mozilla/5.0',
  '192.168.1.6',
  'cookie_value3',
  '8082',
  '{"key": "value3"}',
  datetime.datetime(2024, 9, 24, 7, 46, 8),
  3))

In [114]:
results = np.array(ins_results)
key_dict = {}
for k, kl in enumerate(description):
    key_dict[kl[0]] = k
for i in range(len(split_info)):
    table_name = source_table + '_' + split_info[i]['split_table_name']
    columns_info = split_info[i]['columns']
    drop_query = 'DROP TABLE IF EXISTS %s' % table_name
    cur1.execute(drop_query)
    if split_info[i]['split_table_name'] != 'relation':
        create_query = 'CREATE TABLE IF NOT EXISTS %s (sid INT, ' % table_name
    else:
        create_query = 'CREATE TABLE IF NOT EXISTS %s (' % table_name
        # 生成随机矩阵
        num_rows = results.shape[0]
        num_columns = len(columns_info) - 1  # 减去 'sid' 列

        # 创建插入数据的矩阵
        insert_data = []

        for col in range(num_columns):
            unique_numbers = set()
            while len(unique_numbers) < num_rows:
                random_num = random.randint(100000000, 999999999)
                unique_numbers.add(random_num)
            # 将生成的随机数添加到 DataFrame 中
            # df[f'sid{col}'] = list(unique_numbers)
            unique_numbers = np.array(list(unique_numbers))
            unique_numbers = unique_numbers.reshape(-1, 1)
            results = np.hstack([results, unique_numbers])
            key_dict[f'sid{col}'] = len(key_dict.keys())
    # for k in columns_info:
    #     create_query += '%s %s(%s), ' % (k['name'], k['type'], k['length'])
    # create_query = create_query[:-2]
    # create_query += ')'
    # create_query
    # print(create_query)
    # cur1.execute(create_query)
source_columns = list(key_dict.keys())
target_columns = {}
for i in range(len(split_info)):
    table_name = source_table + '_' + split_info[i]['split_table_name']
    columns_info = split_info[i]['columns']
    if split_info[i]['split_table_name'] != 'relation':
        cross_columns = ['sid']
    else:
        cross_columns = []
    for k in columns_info:
        match_name = k['name']
        # if match_name in source_columns:
        cross_columns.append(k['name'])
    target_columns[table_name] = cross_columns

# 插入子表
insert_query_dict = {}
for table_name in target_columns:
    insert_query = 'INSERT INTO %s (' % table_name
    columns_list = target_columns[table_name]
    for k in columns_list:
        insert_query += '%s, ' % k
    insert_query = insert_query[:-2] + ') VALUES ('
    for i in range(len(columns_list)):
        insert_query += '%s, '
    insert_query = insert_query[:-2] + ')'
    insert_query_dict[table_name] = insert_query

i = 0
for table_name in target_columns:
    if table_name.find('relation') != -1:
        source_keys = [key_dict['sid']]
    else:
        source_keys = [key_dict[f'sid{i}']]
        i = i + 1
    for k in target_columns[table_name]:
        if k != 'sid':
            source_keys.append(key_dict[k])
    insert_data = results[:, source_keys].tolist()
    insert_query = insert_query_dict[table_name]
    cur1.executemany(insert_query, insert_data)
    conn1.commit()

CREATE TABLE IF NOT EXISTS sada_gdpi_click_dtl_relation (sid bigint(20), sid0 bigint(20), sid1 bigint(20), sid2 bigint(20), sid3 bigint(20))
CREATE TABLE IF NOT EXISTS sada_gdpi_click_dtl_server0 (sid INT, f_srcip VARCHAR(100))
CREATE TABLE IF NOT EXISTS sada_gdpi_click_dtl_server1 (sid INT, f_ua VARCHAR(100), f_dstip VARCHAR(100), f_cookie VARCHAR(100))
CREATE TABLE IF NOT EXISTS sada_gdpi_click_dtl_server2 (sid INT, f_ref VARCHAR(100), f_url VARCHAR(100), f_ts VARCHAR(100), f_ad VARCHAR(100))
CREATE TABLE IF NOT EXISTS sada_gdpi_click_dtl_server3 (sid INT, f_dataid VARCHAR(100), f_json VARCHAR(100), f_update_time VARCHAR(100), f_src_port VARCHAR(100))


In [122]:
query_del = "SELECT * FROM sada_gdpi_click_dtl_dgut_del"
cur2.execute(query_ins)
description = cur2.description
del_results = cur2.fetchall()
len(del_results), del_results[0]

(2,
 (910001,
  '192.168.1.3',
  'Ad2',
  1695201000,
  'http://example.org',
  'http://anotherref.com',
  'Mozilla/5.0',
  '192.168.1.4',
  'cookie_value2',
  '8081',
  '{"key": "value2"}',
  datetime.datetime(2024, 9, 24, 7, 46, 8),
  2))

In [123]:
results = np.array(del_results)
key_dict = {}
for k, kl in enumerate(description):
    key_dict[kl[0]] = k
for i in range(len(split_info)):
    table_name = source_table + '_' + split_info[i]['split_table_name']
    columns_info = split_info[i]['columns']
    # drop_query = 'DROP TABLE IF EXISTS %s' % table_name
    # cur1.execute(drop_query)
    if split_info[i]['split_table_name'] != 'relation':
        create_query = 'CREATE TABLE IF NOT EXISTS %s (sid INT, ' % table_name
    else:
        create_query = 'CREATE TABLE IF NOT EXISTS %s (' % table_name
        # 生成随机矩阵
        num_rows = results.shape[0]
        num_columns = len(columns_info) - 1  # 减去 'sid' 列

        # 创建插入数据的矩阵
        insert_data = []

        for col in range(num_columns):
            unique_numbers = set()
            while len(unique_numbers) < num_rows:
                random_num = random.randint(100000000, 999999999)
                unique_numbers.add(random_num)
            # 将生成的随机数添加到 DataFrame 中
            # df[f'sid{col}'] = list(unique_numbers)
            unique_numbers = np.array(list(unique_numbers))
            unique_numbers = unique_numbers.reshape(-1, 1)
            results = np.hstack([results, unique_numbers])
            key_dict[f'sid{col}'] = len(key_dict.keys())
source_columns = list(key_dict.keys())
target_columns = {}
for i in range(len(split_info)):
    table_name = source_table + '_' + split_info[i]['split_table_name']
    columns_info = split_info[i]['columns']
    if split_info[i]['split_table_name'] != 'relation':
        cross_columns = ['sid']
    else:
        cross_columns = []
    for k in columns_info:
        match_name = k['name']
        # if match_name in source_columns:
        cross_columns.append(k['name'])
    target_columns[table_name] = cross_columns

In [138]:
results

array([[910001, '192.168.1.3', 'Ad2', 1695201000, 'http://example.org',
        'http://anotherref.com', 'Mozilla/5.0', '192.168.1.4',
        'cookie_value2', '8081', '{"key": "value2"}',
        datetime.datetime(2024, 9, 24, 7, 46, 8), 2, 137241341,
        365214756, 867997588, 532204752],
       [910002, '192.168.1.5', 'Ad3', 1695202000, 'http://example.net',
        'http://yetanotherref.com', 'Mozilla/5.0', '192.168.1.6',
        'cookie_value3', '8082', '{"key": "value3"}',
        datetime.datetime(2024, 9, 24, 7, 46, 8), 3, 733306070,
        190752694, 918828685, 909078071]], dtype=object)

In [129]:
query_dict

{'sada_gdpi_click_dtl_relation': 'DELETE FROM sada_gdpi_click_dtl_relation WHERE sid =',
 'sada_gdpi_click_dtl_server0': 'DELETE FROM sada_gdpi_click_dtl_server0 WHERE sid =',
 'sada_gdpi_click_dtl_server1': 'DELETE FROM sada_gdpi_click_dtl_server1 WHERE sid =',
 'sada_gdpi_click_dtl_server2': 'DELETE FROM sada_gdpi_click_dtl_server2 WHERE sid =',
 'sada_gdpi_click_dtl_server3': 'DELETE FROM sada_gdpi_click_dtl_server3 WHERE sid ='}

In [139]:
results

array([[910001, '192.168.1.3', 'Ad2', 1695201000, 'http://example.org',
        'http://anotherref.com', 'Mozilla/5.0', '192.168.1.4',
        'cookie_value2', '8081', '{"key": "value2"}',
        datetime.datetime(2024, 9, 24, 7, 46, 8), 2, 137241341,
        365214756, 867997588, 532204752],
       [910002, '192.168.1.5', 'Ad3', 1695202000, 'http://example.net',
        'http://yetanotherref.com', 'Mozilla/5.0', '192.168.1.6',
        'cookie_value3', '8082', '{"key": "value3"}',
        datetime.datetime(2024, 9, 24, 7, 46, 8), 3, 733306070,
        190752694, 918828685, 909078071]], dtype=object)

In [157]:
target_columns

{'sada_gdpi_click_dtl_relation': ['sid', 'sid0', 'sid1', 'sid2', 'sid3'],
 'sada_gdpi_click_dtl_server0': ['sid', 'f_srcip'],
 'sada_gdpi_click_dtl_server1': ['sid', 'f_ua', 'f_dstip', 'f_cookie'],
 'sada_gdpi_click_dtl_server2': ['sid', 'f_ref', 'f_url', 'f_ts', 'f_ad'],
 'sada_gdpi_click_dtl_server3': ['sid',
  'f_dataid',
  'f_json',
  'f_update_time',
  'f_src_port']}

In [154]:
i = 0
del_querys = []
del_idxs = results[:,key_dict['sid']]

# 删除数据
del_key = 'sid'
    
for table_name in target_columns:
    for idx in del_idxs:
        del_query = 'UPDATE %s SET %s=0 WHERE %s=%s' % (table_name, del_key, del_key, idx)
        del_querys.append(del_query)
del_querys

['UPDATE sada_gdpi_click_dtl_relation SET sid=0 WHERE sid=910001',
 'UPDATE sada_gdpi_click_dtl_relation SET sid=0 WHERE sid=910002',
 'UPDATE sada_gdpi_click_dtl_server0 SET sid=0 WHERE sid=910001',
 'UPDATE sada_gdpi_click_dtl_server0 SET sid=0 WHERE sid=910002',
 'UPDATE sada_gdpi_click_dtl_server1 SET sid=0 WHERE sid=910001',
 'UPDATE sada_gdpi_click_dtl_server1 SET sid=0 WHERE sid=910002',
 'UPDATE sada_gdpi_click_dtl_server2 SET sid=0 WHERE sid=910001',
 'UPDATE sada_gdpi_click_dtl_server2 SET sid=0 WHERE sid=910002',
 'UPDATE sada_gdpi_click_dtl_server3 SET sid=0 WHERE sid=910001',
 'UPDATE sada_gdpi_click_dtl_server3 SET sid=0 WHERE sid=910002']

In [156]:
for del_query in del_querys:
    try:
        cur1.execute(del_query)
    except:
        print("SQL Failed: %s"%del_query)
conn1.commit()

SQL Failed: UPDATE sada_gdpi_click_dtl_server3 SET sid=0 WHERE sid=910001
SQL Failed: UPDATE sada_gdpi_click_dtl_server3 SET sid=0 WHERE sid=910002


In [153]:
del_query

'UPDATE table_name sada_gdpi_click_dtl_relation SET sid=0 WHERE sid=910001'

In [141]:
del_querys

['DELETE FROM sada_gdpi_click_dtl_relation WHERE sid = 910001;',
 'DELETE FROM sada_gdpi_click_dtl_relation WHERE sid = 910001;',
 'DELETE FROM sada_gdpi_click_dtl_server0 WHERE sid = 137241341;',
 'DELETE FROM sada_gdpi_click_dtl_server0 WHERE sid = 137241341;',
 'DELETE FROM sada_gdpi_click_dtl_server1 WHERE sid = 365214756;',
 'DELETE FROM sada_gdpi_click_dtl_server1 WHERE sid = 365214756;',
 'DELETE FROM sada_gdpi_click_dtl_server2 WHERE sid = 867997588;',
 'DELETE FROM sada_gdpi_click_dtl_server2 WHERE sid = 867997588;',
 'DELETE FROM sada_gdpi_click_dtl_server3 WHERE sid = 532204752;',
 'DELETE FROM sada_gdpi_click_dtl_server3 WHERE sid = 532204752;']

In [134]:
del_querys

['DELETE FROM sada_gdpi_click_dtl_relation WHERE sid = 910001;',
 'DELETE FROM sada_gdpi_click_dtl_relation WHERE sid = 910001;',
 'DELETE FROM sada_gdpi_click_dtl_server0 WHERE sid = 137241341;',
 'DELETE FROM sada_gdpi_click_dtl_server0 WHERE sid = 137241341;',
 'DELETE FROM sada_gdpi_click_dtl_server1 WHERE sid = 365214756;',
 'DELETE FROM sada_gdpi_click_dtl_server1 WHERE sid = 365214756;',
 'DELETE FROM sada_gdpi_click_dtl_server2 WHERE sid = 867997588;',
 'DELETE FROM sada_gdpi_click_dtl_server2 WHERE sid = 867997588;',
 'DELETE FROM sada_gdpi_click_dtl_server3 WHERE sid = 532204752;',
 'DELETE FROM sada_gdpi_click_dtl_server3 WHERE sid = 532204752;']

In [82]:
del_query

'DELETE FROM sada_gdpi_click_dtl_relation WHERE sid = 110001;'

In [45]:

for i in range(len(split_info)):
    table_name = source_table + '_' + split_info[i]['split_table_name']
    columns_info = split_info[i]['columns']
    drop_query = 'DROP TABLE IF EXISTS %s' % table_name
    cur1.execute(drop_query)
    if split_info[i]['split_table_name'] != 'relation':
        create_query = 'CREATE TABLE IF NOT EXISTS %s (sid INT, ' % table_name
    else:
        create_query = 'CREATE TABLE IF NOT EXISTS %s (' % table_name
        #生成随机矩阵
        num_rows = results.shape[0]
        num_columns = len(columns_info) - 1  # 减去 'sid' 列

        # 创建插入数据的矩阵
        insert_data = []

        for col in range(num_columns):
            unique_numbers = set()
            while len(unique_numbers) < num_rows:
                random_num = random.randint(100000000, 999999999)
                unique_numbers.add(random_num)
            # 将生成的随机数添加到 DataFrame 中
            # df[f'sid{col}'] = list(unique_numbers)
            unique_numbers = np.array(list(unique_numbers))
            unique_numbers = unique_numbers.reshape(-1, 1)
            results = np.hstack([results, unique_numbers])
            key_dict[f'sid{col}'] = len(key_dict.keys())
    for k in columns_info:
        create_query += '%s %s(%s), '%(k['name'], k['type'], k['length'])
    create_query = create_query[:-2]
    create_query += ')'
    create_query
    print(create_query)
    cur1.execute(create_query)


CREATE TABLE IF NOT EXISTS sada_gdpi_click_dtl_relation (sid bigint(20), sid0 bigint(20), sid1 bigint(20), sid2 bigint(20), sid3 bigint(20))
CREATE TABLE IF NOT EXISTS sada_gdpi_click_dtl_server0 (sid INT, f_srcip VARCHAR(100))
CREATE TABLE IF NOT EXISTS sada_gdpi_click_dtl_server1 (sid INT, f_ua VARCHAR(100), f_dstip VARCHAR(100), f_cookie VARCHAR(100))
CREATE TABLE IF NOT EXISTS sada_gdpi_click_dtl_server2 (sid INT, f_ref VARCHAR(100), f_url VARCHAR(100), f_ts VARCHAR(100), f_ad VARCHAR(100))
CREATE TABLE IF NOT EXISTS sada_gdpi_click_dtl_server3 (sid INT, f_dataid VARCHAR(100), f_json VARCHAR(100), f_update_time VARCHAR(100), f_src_port VARCHAR(100))


In [26]:

source_columns = list(key_dict.keys())
target_columns = {}
for i in range(len(split_info)):
    table_name = source_table + '_' + split_info[i]['split_table_name']
    columns_info = split_info[i]['columns']
    if split_info[i]['split_table_name'] != 'relation':
        cross_columns = ['sid']
    else:
        cross_columns = []
    for k in columns_info:
        match_name = k['name']
        #if match_name in source_columns:
        cross_columns.append(k['name'])
    target_columns[table_name] = cross_columns


In [28]:

# 插入子表
insert_query_dict = {}
for table_name in target_columns:
    insert_query = 'INSERT INTO %s ('%table_name
    columns_list = target_columns[table_name]
    for k in columns_list:
        insert_query += '%s, '%k
    insert_query = insert_query[:-2] + ') VALUES ('
    for i in range(len(columns_list)):
        insert_query += '%s, '
    insert_query = insert_query[:-2]+')'
    insert_query_dict[table_name] = insert_query


In [48]:

i = 0;
for table_name in target_columns:
    if table_name.find('relation') != -1:
        source_keys = [key_dict['sid']];
    else:
        source_keys = [key_dict[f'sid{i}']];
        i = i + 1;
    for k in target_columns[table_name]:
        if k !='sid':
            source_keys.append(key_dict[k])
    insert_data = results[:, source_keys].tolist()
    insert_query = insert_query_dict[table_name]
    cur1.executemany(insert_query, insert_data)
    conn1.commit()
    # break


In [41]:
key_dict

{'sid': 0,
 'f_srcip': 1,
 'f_ad': 2,
 'f_ts': 3,
 'f_url': 4,
 'f_ref': 5,
 'f_ua': 6,
 'f_dstip': 7,
 'f_cookie': 8,
 'f_src_port': 9,
 'f_json': 10,
 'f_update_time': 11,
 'f_dataid': 12,
 'sid0': 14,
 'sid1': 15,
 'sid2': 16,
 'sid3': 17}

In [40]:
results[:, source_keys]

IndexError: index 17 is out of bounds for axis 1 with size 17

In [35]:
key_dict

{'sid': 0,
 'f_srcip': 1,
 'f_ad': 2,
 'f_ts': 3,
 'f_url': 4,
 'f_ref': 5,
 'f_ua': 6,
 'f_dstip': 7,
 'f_cookie': 8,
 'f_src_port': 9,
 'f_json': 10,
 'f_update_time': 11,
 'f_dataid': 12,
 'sid0': 14,
 'sid1': 15,
 'sid2': 16,
 'sid3': 17}

In [33]:
insert_query

'INSERT INTO sada_gdpi_click_dtl_relation (sid, sid0, sid1, sid2, sid3) VALUES (%s, %s, %s, %s, %s)'

In [34]:
insert_data

[[1,
  '224.93.65.63',
  'a1dc2c7141eda1422e6b33c62296a0a24c014e21460cc4168d581a',
  1715270352010,
  'ODdmNzRmNjM0YmY0ODgxNDFlNmUxZmFiYTFjYjc5M2Q1YmFiMDIxYmIyODdmMDAwMzFhYjY3ZDhiMzNkNDA2ZA==',
  '',
  'com.sankuai.meituan/1201201404 (Linux; U; Android 22; zh_CN; Redmi K30 Pro; Build/RKQ1.201826.012; C',
  '103.37.155.60',
  '',
  'A9D8',
  '',
  datetime.datetime(2024, 8, 19, 0, 2, 40),
  6780877879247444511,
  340787200,
  362807296,
  717488128,
  479723520],
 [15,
  '58.40.147.146',
  'bdde9edc1abcf7e062151e6b8c576bf80ac3',
  1715270410101,
  'http://123.184.24.196:8010/v1/resource/1535464238-1-30232.m4s?build=76601101&buvid=Y248B32BAA43B5622',
  '',
  'Bilibili Freedoooooom/MarkII',
  '123.184.24.196',
  '',
  'D7B',
  '',
  datetime.datetime(2024, 8, 19, 0, 2, 40),
  8623023287988953196,
  246415402,
  306970653,
  569638953,
  729808920],
 [16,
  '224.93.210.228',
  'f18b287849eda7481c6571243f8c36d89f569529b24cda634bce',
  1715270410101,
  'http://pull-l3.douyincdn.com/pull-l3.d